<a href="https://colab.research.google.com/github/jp-prud/PDI/blob/main/Filtragem_no_Dom%C3%ADnio_da_Frequ%C3%AAncia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Usando a DFT e FFT da OpenCV

Mostrando a magnitude da TF

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('Fig0424(a)(rectangle).tif',0)
f = np.fft.fft2(img)
fshift = np.fft.fftshift(f)
magnitude_spectrum = (20*np.log(np.abs(fshift) + 0.00001)/255)
magnitude_spectrum = np.array(magnitude_spectrum, dtype=np.float32)

plt.subplot(121),plt.imshow(img, cmap = 'gray')
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(magnitude_spectrum, cmap = 'gray')
plt.title('Magnitude Spectrum'), plt.xticks([]), plt.yticks([])
plt.show()

#mostrando a imagem de entrada
cv2.imshow('in', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

#mostrando a magnitude da TF
cv2.imshow('magnitude_spectrum', magnitude_spectrum)
cv2.waitKey(0)
cv2.destroyAllWindows()

Criando espectro de magnitude

In [ ]:
img = cv2.imread('t1.jpg',0)
image_f32 = img.astype(np.float32)# convert from uint8 into float32
dft = cv2.dft(image_f32, flags = cv2.DFT_COMPLEX_OUTPUT)
fft_shift = np.fft.fftshift(dft) #aplica FFT e posteriormente a desloca - dica é usar a função presente na numpy
magnitude_spectrum = (20*np.log(np.abs(fft_shift) + 0.00001)/255) # cria o espectro de magnitude

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Filtro PB ideal

In [ ]:
img = cv2.imread('t1.jpg',0)
f = np.fft.fft2(img)
fshift = np.fft.fftshift(f)

image_f32 = img.astype(np.float32)# convert from uint8 into float32
dft = cv2.dft(image_f32, flags = cv2.DFT_COMPLEX_OUTPUT)
magnitude_spectrum = (20*np.log(np.abs(fshift) + 0.00001)/255) # cria o espectro de magnitude

rows, cols = img.shape
crow, ccol = rows // 2, cols // 2
mask = np.zeros((rows, cols, 2), np.uint8)  # H(u,v)

# Defina o raio de corte (valor inteiro)
raio = 20

# Crie um círculo com valor 1 dentro do raio de corte
y, x = np.ogrid[-crow:rows - crow, -ccol:cols - ccol]
mascara_circular = x**2 + y**2 <= raio**2
mask[mascara_circular] = 1

# Inverta a máscara para manter as baixas frequências e atenuar as altas frequências
mask_invertida = 1 - mask

# Aplique a máscara ao espectro de frequência
dft_filtered = dft * mask_invertida

# Para visualização, você pode calcular o espectro de magnitude do resultado
magnitude_spectrum_filtered = np.log(1 + np.abs(np.fft.fftshift(dft_filtered)))

# Aplicar a máscara ao espectro de frequência
fshift = dft * mask_invertida

# Aplicar a DFT inversa (IDFT)
f_ishift = np.fft.ifftshift(fshift)
img_back = cv2.idft(f_ishift)
img_back = cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])

img_back = np.array(img_back, dtype=np.float32)

img_back = cv2.flip(img_back, 0)

print(np.argmax(img_back))
plt.subplot(121),plt.imshow(magnitude_spectrum, cmap = 'gray')
plt.title('Magnitude Spectrum'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(img_back, cmap = 'gray')
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.show()

filtered_img = np.abs(img_back)  # Aplicar o absoluto
filtered_img -= filtered_img.min()   # Subtrair o valor do menor pixel
filtered_img = (filtered_img * 255 / filtered_img.max()).astype(np.uint8)  # Escalar e fazer um cast para 8 bits

# caso use o colab, utilize o patch corretamente
cv2.imshow('img_back', filtered_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Criação da filtro PA -- Código com problema

In [ ]:
img = cv2.imread('Fig0424(a)(rectangle).tif',0)

image_f32 = img.astype(np.float32)# convert from uint8 into float32
dft = cv2.dft(image_f32, flags = cv2.DFT_COMPLEX_OUTPUT)
rows, cols = img.shape
crow, ccol = rows // 2, cols // 2

# Crie uma máscara inicializada com zeros
mask = np.zeros((rows, cols, 2), np.uint8)

# Defina o raio de corte (valor inteiro)
raio = 20

# Crie um círculo com valor 1 fora do raio de corte
y, x = np.ogrid[-crow:rows - crow, -ccol:cols - ccol]
mascara_circular = x**2 + y**2 > raio**2
mask[mascara_circular] = 1

# Aplicar a máscara ao espectro de frequência
fshift = dft * mask

# Aplicar a DFT inversa (IDFT)
f_ishift = np.fft.ifftshift(fshift)
img_back = cv2.idft(f_ishift)
img_back = cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])

img_back = np.array(img_back, dtype=np.float32)

img_back = cv2.flip(img_back, 0)

magnitude_spectrum = (20*np.log(np.abs(fshift) + 0.00001)/255)
magnitude_spectrum = np.array(magnitude_spectrum, dtype=np.float32)


plt.subplot(121),plt.imshow(img_back, cmap = 'gray')
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(magnitude_spectrum, cmap = 'gray')
plt.title('Magnitude Spectrum'), plt.xticks([]), plt.yticks([])
plt.show()

# Normalizar a imagem para o intervalo 0-255
filtered_img = np.abs(img_back)  # Aplicar o absoluto
filtered_img -= filtered_img.min()   # Subtrair o valor do menor pixel
filtered_img = (filtered_img * 255 / filtered_img.max()).astype(np.uint8)  # Escalar e fazer um cast para 8 bits


cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow('filtered_img', filtered_img)
cv2.waitKey(0)
cv2.destroyAllWindows()